# Creating your first Manus Task

> Prerequisites: Sign up for an account at https://manus.im/ and create your first API Key

Welcome to the first step in your Manus journey! This notebook is all about getting you hands-on with the API as quickly as possible. We'll skip the long lectures and jump right into the core mechanics of creating and managing tasks programmatically.

By the end of this session, you'll have a solid foundation for building more advanced automations. Here’s a quick look at what we'll cover:

1.  **Connect to Manus**: We'll start by loading your API key and making a test call to ensure everything is set up correctly.
2.  **Create Your First Task**: You’ll give Manus a simple instruction and learn how to interpret the initial response.
3.  **Master the Task Lifecycle**: We'll see how to track a task from a "running" state to "pending" or "completed" by polling the API for updates.
4.  **Leverage Connectors**: We'll finish with a quick look at how you can give Manus access to your other tools, like Notion, to perform more complex workflows.

Let's get started.

## Setting Up Your Environment

Before interacting with Manus, your environment needs valid credentials and basic configuration.

In this section, you will load your API key, validate connectivity, and make a simple test request to confirm everything is working.

### Authenticate Your API Key

First, let's load your API key from the .env file in your repository root and set up the basic configuration we'll need for all our API calls.

In [2]:
import os
import requests
from dotenv import load_dotenv
from rich import print

# Load API key from .env file in repository root
load_dotenv()

# Configuration
API_KEY = os.getenv("MANUS_API_KEY")
BASE_URL = "https://api.manus.ai/v1"

# Validate API key is loaded
if not API_KEY:
    raise ValueError("⚠️  Please set your MANUS_API_KEY in the .env file at the repository root")

print(f"✓ API key loaded successfully!\n✓ Base URL: {BASE_URL}")

/Users/ivanleo-work/Documents/coding/manus-api-webhook-workshop/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


✓ API key loaded successfully!
✓ Base URL: https://api.manus.ai/v1

### Validate Your Connection to the Manus API

Now let's make a simple test request to ensure our API key is valid and we can successfully connect to the Manus API. We'll use the Files endpoint for this quick check.

In [3]:
url = f"{BASE_URL}/files"
response = requests.get(url, headers={"API_KEY": API_KEY})

print(response.json())

{
    'object': 'list',
    'data': [
        {
            'id': 'file-kTn4gC6RUS4uZXisjvpvNq',
            'object': 'file',
            'bytes': '73350',
            'created_at': '1763343047',
            'expires_at': '1763515847',
            'filename': 'rick_and_morty_characters.json',
            'status': 'uploaded',
            'content_type': 'application/json'
        },
        {
            'id': 'file-RLdVRHwbEyvvefNgmEzNDq',
            'object': 'file',
            'bytes': '73350',
            'created_at': '1763342721',
            'expires_at': '1763515522',
            'filename': 'rick_and_morty_characters.json',
            'status': 'uploaded',
            'content_type': 'application/json'
        }
    ]
}

If you see a response like {'object': 'list'}, it means your connection is working perfectly! You're now ready to create your first real Manus task and see how the API behaves when executing a simple instruction.

## Creating Your First Task

Now that our environment is authenticated, let's create our first Manus task. This is where we'll see Manus in action, taking a simple instruction and providing a response.

### Our First Request

Let's start with a straightforward mathematical query. We'll ask Manus to solve "2 + 2" and observe how it handles this simple request.

In [4]:
# Create a simple task
headers = {
    "API_KEY": API_KEY,
    "Content-Type": "application/json"
}

data = {
    "prompt": "What is 2 + 2? Please provide a brief answer.",
    "agentProfile": "manus-1.5"
}

response = requests.post(
    f"{BASE_URL}/tasks",
    headers=headers,
    json=data
)
response.raise_for_status()

task = response.json()

print(task)

{
    'task_id': '2bWswULe9aopi4Y26TwHju',
    'task_title': 'What is 2 plus 2',
    'task_url': 'https://manus.im/app/2bWswULe9aopi4Y26TwHju'
}

### Understand the Task Response

When you create a task, Manus returns several key pieces of information that are essential for tracking and managing your request:

For this basic mathematical question, Manus will detect the simplicity and switch to Chat mode, providing an immediate response without unnecessary computational overhead. 

The key values you just extracted are:

1. task_id: A unique identifier that you'll use to reference this specific task
2. task_url: A direct link to view the task execution in the Manus web interface
3. task_title: An automatically generated title that summarizes your request

### A Quick Look at Agent Profiles

> To read more about these two different agent profiles, you can check out our [announcement article here](https://manus.im/blog/manus-1.5-release)

Manus offers different agent profiles optimized for different use cases. Let's understand the two main options:

1. manus-1.5-lite: Our full-featured agent with maximum capability. This is recommended for tasks that require more complex reasoning and agentic capabilities.
2. manus-1.5: A streamlined version that's optimized for cost efficiency while still maintaining strong performance on most tasks.

Our new agent architecture delivers nearly four times faster performance—tasks that once took 15 minutes now complete in under 4 minutes—while achieving 15% better task quality and 6% higher user satisfaction.

The agent also comes with expanded capabilities like:

- Unlimited Context Management: Maintains coherence across longer conversations
- Web Development Capabilities: Can create entire websites with database and backend server functionality

For our simple math example, either profile would work perfectly. As we progress to more complex tasks, you'll see when to choose each profile based on your specific needs.

## Managing Asynchronous Tasks

Tasks in Manus run asynchronously, meaning they execute in the background and can take time to complete. They might transition through multiple states or even pause to wait for your input. 

This section will teach you how to programmatically track a task's progress and interact with it throughout its lifecycle.

### Polling Task Status

The most straightforward way to monitor a task is by "polling" the API—periodically asking for an update. 

Let's create a new task designed to pause and wait for our input. This will allow us to observe the task transition from a running state to a pending state.

We have four primary states to watch for:

1. running: The task is actively being processed.
2. pending: Manus has finished a step and is waiting for your input before continuing.
3. completed: The task has finished successfully.
4. error: Manus has encountered an unrecoverable error.

Let's create a task that will look up the weather and then ask for our input.

In [5]:
# Create a task designed to enter a pending state
data = {
    "prompt": "Look up the weather in New York City next week and then ask me a question about it. Make sure that you wait for my response before you continue",
    "agentProfile": "manus-1.5-lite",
    "taskMode": "agent"
}

response = requests.post(
    f"{BASE_URL}/tasks",
    headers=headers,
    json=data
)
response.raise_for_status()

task_response = response.json()
task_id = task_response["task_id"]
print("✓ Task created successfully!\n")
print(task_response)

✓ Task created successfully!

{
    'task_id': 'Q64L2WtH5X6drABXoZMPnY',
    'task_title': 'Weather in New York City Next Week?',
    'task_url': 'https://manus.im/app/Q64L2WtH5X6drABXoZMPnY'
}

Now, let's write a simple loop to poll the task's status every 20 seconds until it's no longer running.

In [7]:
import time

POLLING_INTERVAL = 60

while True:
    url = f"{BASE_URL}/tasks/{task_id}"
    headers = {"API_KEY": API_KEY}
    response = requests.get(url, headers=headers)
    task = response.json()
    task_url = task["metadata"]["task_url"]

    if task["status"] != "running":
        print(f"\n✓ Task {task_id} is now {task['status']}!")
        break

    print(f"Task is still running... waiting {POLLING_INTERVAL}s. Check progress: {task_url}")
    time.sleep(POLLING_INTERVAL)

print("\nFinal task details:")
print(task)

Task is still running... waiting 60s. Check progress: https://manus.im/app/Q64L2WtH5X6drABXoZMPnY

✓ Task Q64L2WtH5X6drABXoZMPnY is now pending!

Final task details:

{
    'id': 'Q64L2WtH5X6drABXoZMPnY',
    'object': 'task',
    'created_at': '1763501439',
    'updated_at': '1763501476',
    'status': 'pending',
    'model': 'manus-1.5-lite-agent',
    'metadata': {'task_url': 'https://manus.im/app/Q64L2WtH5X6drABXoZMPnY'},
    'output': [
        {
            'id': 'mQsyquQWbDTMouUrzBLGKM',
            'status': 'completed',
            'role': 'user',
            'type': 'message',
            'content': [
                {
                    'type': 'output_text',
                    'text': 'Look up the weather in New York City next week and then ask me a question about it. 
Make sure that you wait for my response before you continue'
                }
            ]
        },
        {
            'id': 'b4gPhsY9T5fQsUrV2f0CId',
            'status': 'completed',
            'role': 'assistant',
            'type': 'message',
            'content': [
                {
                    'type': 'output_text',
                    'text': 'Understood. I will look up the weather forecast for New York City next week and then 
ask you a question about it.'
                }
            ]
        },
        {
            'id': 'deSwDsLgHs9jDVgquRkWZq',
            'status': 'completed',
            'role': 'assistant',
            'type': 'message',
            'content': [
                {
                    'type': 'output_text',
                    'text': 'I have looked up the weather forecast for New York City for the next week (November 
19th to November 25th).\n\nThe forecast shows a range of temperatures and precipitation chances:\n\n| Day | High 
(°F) | Low (°F) | Chance of Precipitation |\n| :--- | :--- | :--- | :--- |\n| Wednesday | 46 | 37 | 64% |\n| 
Thursday | 47 | 43 | 5% |\n| Friday | 53 | 43 | 0% |\n| Saturday | 50 | 39 | 0% |\n| Sunday | 52 | 44 | 8% |\n| 
Monday | 55 | 43 | 0% |\n| Tuesday | 53 | 43 | 10% |\n\nBased on this, I have a question for you:\n\n**If you were 
planning an outdoor activity for next week, which day would you choose to minimize the chance of rain, and why?**'
                }
            ]
        }
    ],
    'credit_usage': 11
}

As you can see from the output, the task status is now pending. This means Manus has completed its initial instruction (looking up the weather) and is now waiting for you to provide the next input, just as we asked. 

The output field contains the full conversation history, including the question Manus is asking. 

When you use the Manus API to create more complex queries, you'll need to do things like download reports and other generated content, all of which are returned in the `output` array.

### Continuing a Task

To continue the conversation, we simply create a new task but include the taskId of the conversation we want to continue. This tells Manus to add our new prompt to the existing session instead of starting a new one.

Let's answer the question Manus asked.

In [8]:
# Continue the existing task
data = {
    "prompt": "I'd probably choose probably friday or saturday",
    "agentProfile": "manus-1.5-lite",
    "taskMode": "agent",
    "taskId": task_id  # This is the key to continuing the conversation
}

response = requests.post(
    f"{BASE_URL}/tasks",
    headers=headers,
    json=data
)
response.raise_for_status()

task_response = response.json()
print("✓ Follow-up prompt sent successfully!\n")
print(task_response)

✓ Follow-up prompt sent successfully!

{
    'task_id': 'Q64L2WtH5X6drABXoZMPnY',
    'task_title': 'Weather in New York City Next Week',
    'task_url': 'https://manus.im/app/Q64L2WtH5X6drABXoZMPnY'
}

Now, we can use our same polling loop to wait for the task to fully complete.

In [9]:
POLLING_INTERVAL = 30

while True:
    url = f"{BASE_URL}/tasks/{task_id}"
    headers = {"API_KEY": API_KEY}
    response = requests.get(url, headers=headers)
    task = response.json()
    task_url = task["metadata"]["task_url"]

    if task["status"] != "running":
        print(f"\n✓ Task {task_id} is now {task['status']}!")
        break

    print(f"Task is still running... waiting {POLLING_INTERVAL}s. Check progress: {task_url}")
    time.sleep(POLLING_INTERVAL)

print("\nFinal task details:")
print(task)

Task is still running... waiting 30s. Check progress: https://manus.im/app/Q64L2WtH5X6drABXoZMPnY

✓ Task Q64L2WtH5X6drABXoZMPnY is now completed!

Final task details:

{
    'id': 'Q64L2WtH5X6drABXoZMPnY',
    'object': 'task',
    'created_at': '1763501439',
    'updated_at': '1763501562',
    'status': 'completed',
    'model': 'manus-1.5-lite-agent',
    'metadata': {'task_url': 'https://manus.im/app/Q64L2WtH5X6drABXoZMPnY'},
    'output': [
        {
            'id': 'mQsyquQWbDTMouUrzBLGKM',
            'status': 'completed',
            'role': 'user',
            'type': 'message',
            'content': [
                {
                    'type': 'output_text',
                    'text': 'Look up the weather in New York City next week and then ask me a question about it. 
Make sure that you wait for my response before you continue'
                }
            ]
        },
        {
            'id': 'b4gPhsY9T5fQsUrV2f0CId',
            'status': 'completed',
            'role': 'assistant',
            'type': 'message',
            'content': [
                {
                    'type': 'output_text',
                    'text': 'Understood. I will look up the weather forecast for New York City next week and then 
ask you a question about it.'
                }
            ]
        },
        {
            'id': 'deSwDsLgHs9jDVgquRkWZq',
            'status': 'completed',
            'role': 'assistant',
            'type': 'message',
            'content': [
                {
                    'type': 'output_text',
                    'text': 'I have looked up the weather forecast for New York City for the next week (November 
19th to November 25th).\n\nThe forecast shows a range of temperatures and precipitation chances:\n\n| Day | High 
(°F) | Low (°F) | Chance of Precipitation |\n| :--- | :--- | :--- | :--- |\n| Wednesday | 46 | 37 | 64% |\n| 
Thursday | 47 | 43 | 5% |\n| Friday | 53 | 43 | 0% |\n| Saturday | 50 | 39 | 0% |\n| Sunday | 52 | 44 | 8% |\n| 
Monday | 55 | 43 | 0% |\n| Tuesday | 53 | 43 | 10% |\n\nBased on this, I have a question for you:\n\n**If you were 
planning an outdoor activity for next week, which day would you choose to minimize the chance of rain, and why?**'
                }
            ]
        },
        {
            'id': 'd0M4JvMtodDAE7Qcpj0w00',
            'status': 'completed',
            'role': 'user',
            'type': 'message',
            'content': [{'type': 'output_text', 'text': "I'd probably choose probably friday or saturday"}]
        },
        {
            'id': 'Mx3uF7XquCW3nRQAtSDmRf',
            'status': 'completed',
            'role': 'assistant',
            'type': 'message',
            'content': [
                {
                    'type': 'output_text',
                    'text': 'That is a great choice! Both Friday and Saturday have a 0% chance of precipitation, 
making them the best days for an outdoor activity based on the forecast.\n\n| Day | High (°F) | Low (°F) | Chance 
of Precipitation |\n| :--- | :--- | :--- | :--- |\n| **Friday** | **53** | **43** | **0%** |\n| **Saturday** | 
**50** | **39** | **0%** |\n\nThe task is now complete. Let me know if you have any other questions or tasks!'
                }
            ]
        }
    ],
    'credit_usage': 16
}

And there you have it! The task status is now completed. You have successfully managed a task through its entire lifecycle, from creation to pending, and finally to completion.

## Conclusion

Great work! You've successfully navigated the core mechanics of the Manus API.

In this notebook, we covered the essentials:

1. Authenticating your connection to the Manus API.
2. Creating your first task and understanding the response.
3. Managing the entire asynchronous task lifecycle, from running to pending and finally to completed.
4. Continuing a conversation within the same task.

In the next notebook, we'll build on these fundamentals by learning how to work with files. You'll discover how to provide Manus with data it needs to reason effectively, whether through direct uploads, URL references, or base64 encoding. 